## 50개 가계정보를 가져와 csv로 만들기

In [54]:
from bs4 import BeautifulSoup 
from urllib.request import urlopen

url_base = 'http://www.chicagomag.com'
url_sub = '/Chicago-Magazine/November-2012/Best-Sandwiches-Chicago/'
url = url_base + url_sub

html = urlopen(url)
soup = BeautifulSoup(html, "html.parser")

In [3]:
#가계정보 가져오기

In [55]:
tmp_one = soup.find_all('div', 'sammy')[0]

In [56]:
from urllib.parse import urljoin
import re

In [57]:
rank = []
main_menu = []
cafe_name = []
url_add = []

list_soup = soup.find_all('div', 'sammy')

for item in list_soup:
    rank.append(item.find(class_='sammyRank').get_text())
    
    tmp_string = item.find(class_='sammyListing').get_text()

    main_menu.append(re.split(('\n|\r\n'), tmp_string)[0])
    cafe_name.append(re.split(('\n|\r\n'), tmp_string)[1])
    
    url_add.append(urljoin(url_base, item.find('a')['href']))

In [58]:
import pandas as pd
import numpy as np

In [59]:
df = pd.DataFrame({'rank':rank, 'menu':main_menu, 'name':cafe_name, 'url':url_add})

In [60]:
df.head()

,rank,menu,name,url
0,1,BLT,Old Oak Tap,http://www.chicagomag.com/Chicago-Magazine/Nov...
1,2,Fried Bologna,Au Cheval,http://www.chicagomag.com/Chicago-Magazine/Nov...
2,3,Woodland Mushroom,Xoco,http://www.chicagomag.com/Chicago-Magazine/Nov...
3,4,Roast Beef,Al’s Deli,http://www.chicagomag.com/Chicago-Magazine/Nov...
4,5,PB&L,Publican Quality Meats,http://www.chicagomag.com/Chicago-Magazine/Nov...


In [61]:
df = df.set_index('rank')

In [62]:
df.to_csv('data.csv')

## csv 파일을 이용해 mysql에 데이터를 삽입해 봅시다.

In [63]:
import csv
import pymysql

In [132]:
mydb = pymysql.connect(host='localhost',
    user='iotuser',
    passwd='iot123',
    db='iot_db')
cursor = mydb.cursor()
mydb.commit()

In [139]:
f = open('./data.csv', 'r', encoding='utf-8')
csv_data = csv.reader(f)

In [140]:
cnt = 1
for row in csv_data:
    rank_col = (row[0])
    menu_col = (row[1])
    name_col = (row[2])
    url_col = (row[3])
    print(rank_col)
    print(menu_col)
    print(name_col)
    print(url_col)
    sql = """
          insert into hamburger (`rank`, `menu`, `name`,`url`) 
          values (%s, %s, %s, %s)
          """
    if(cnt > 1):
        cursor.execute(sql, (int(rank_col), menu_col, name_col, url_col))
    else:
        cnt+=1

rank
menu
name
url
1
BLT
Old Oak Tap
http://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/
2
Fried Bologna
Au Cheval
http://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Au-Cheval-Fried-Bologna/
3
Woodland Mushroom
Xoco
http://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Xoco-Woodland-Mushroom/
4
Roast Beef
Al’s Deli
http://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Als-Deli-Roast-Beef/
5
PB&L
Publican Quality Meats
http://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Publican-Quality-Meats-PB-L/
6
Belgian Chicken Curry Salad
Hendrickx Belgian Bread Crafter
http://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Hendrickx-Belgian-Bread-Crafter-Belgian-Chicken-Curry-Salad/
7
Lobster Roll
Acadia
http://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Acadia-Lob

In [141]:
mydb.commit()

In [142]:
#close the connection to the database.
cursor.close()
print ("Done")

Done
